In [23]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient


#AzureAIServies endpoint&key
endpoint = "https://globalaibootcamp-s1-01.cognitiveservices.azure.com/"
key = "75b7320edf304e52981f453cfda924ad"

#사용할 AI 모델 이름
model_id = "first_model"

#
formUrl = "https://gabs101.blob.core.windows.net/gabs101/예제1.pdf"

document_analysis_client = DocumentAnalysisClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

# Make sure your document's type is included in the list of document types the custom model can analyze
poller = document_analysis_client.begin_analyze_document_from_url(model_id, formUrl)
result = poller.result()

for idx, document in enumerate(result.documents):
    print("--------Analyzing document #{}--------".format(idx + 1))
    print("Document has type {}".format(document.doc_type))
    print("Document has confidence {}".format(document.confidence))
    print("Document was analyzed by model with ID {}".format(result.model_id))
    for name, field in document.fields.items():
        field_value = field.value if field.value else field.content
        print("......found field of type '{}' with value '{}' and with confidence {}".format(field.value_type, field_value, field.confidence))


# iterate over tables, lines, and selection marks on each page
for page in result.pages:
    print("\nLines found on page {}".format(page.page_number))
    for line in page.lines:
        print("...Line '{}'".format(line.content.encode('utf-8')))
    for word in page.words:
        print(
            "...Word '{}' has a confidence of {}".format(
                word.content.encode('utf-8'), word.confidence
            )
        )
    for selection_mark in page.selection_marks:
        print(
            "...Selection mark is '{}' and has a confidence of {}".format(
                selection_mark.state, selection_mark.confidence
            )
        )

for i, table in enumerate(result.tables):
    print("\nTable {} can be found on page:".format(i + 1))
    for region in table.bounding_regions:
        print("...{}".format(i + 1, region.page_number))
    for cell in table.cells:
        print(
            "...Cell[{}][{}] has content '{}'".format(
                cell.row_index, cell.column_index, cell.content.encode('utf-8')
            )
        )
print("-----------------------------------")


HttpResponseError: (InvalidRequest) Invalid request.
Code: InvalidRequest
Message: Invalid request.
Inner error: {
    "code": "InvalidManagedIdentity",
    "message": "The managed identity configuration is invalid: Managed identity is not enabled for the current resource."
}

In [4]:
%pip install azure-storage-blob azure-identity azure-ai-documentintelligence

Note: you may need to restart the kernel to use updated packages.


In [56]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.storage.blob import BlobServiceClient, BlobClient
import os

output_directory = "C:/Users/parkm/Desktop/"

def doing(file_name):

    # Azure Document Intelligence  서비스의 엔드포인트와 키, 모델 이름
    form_recognizer_endpoint = "https://globalaibootcamp-s1-01.cognitiveservices.azure.com/"
    form_recognizer_key = "75b7320edf304e52981f453cfda924ad"
    model_id = "first_model"

    # Azure Blob Storage 계정 정보
    blob_account_name = "gabs101"
    blob_account_key = "BP3TFJ+rMuxxCq/zDutEjqVCuuAfjijzEaUEe07wwsofI2XTpZ/+w0hRxMEV2wQYYIlCDvV7Ubez+AStoDXWeA=="

    # Blob Storage 클라이언트 생성
    blob_service_client = BlobServiceClient(
        account_url=f"https://{blob_account_name}.blob.core.windows.net",
        credential=blob_account_key
    )

    # Blob의 일반 링크
    blob_url = f'https://gabs101.blob.core.windows.net/gabs101/{file_name}'

    # Blob 다운로드
    blob_client = BlobClient.from_blob_url(blob_url, credential=blob_account_key)
    blob_data = blob_client.download_blob().readall()

    # Document Analysis 클라이언트 생성
    document_analysis_client = DocumentAnalysisClient(
        endpoint=form_recognizer_endpoint, credential=AzureKeyCredential(form_recognizer_key)
    )



    # Make sure your document's type is included in the list of document types the custom model can analyze
    poller = document_analysis_client.begin_analyze_document(model_id, blob_data)
    result = poller.result()

    txt_file_name = os.path.join(output_directory, f"{os.path.splitext(file_name)[0]}.txt")

    for document in result.documents:
        with open(txt_file_name, "w") as file:
            for name, field in document.fields.items():
                field_value = field.value if field.value else field.content
                print("{}:'{}'".format(name, field_value))
                file.write(f"{name}: '{field_value}'\n")

   

In [59]:
doing("예제1.pdf")

learnt:'Thank u so much 여 러 분'
reason:'걍 참 가 했 음'
satis_survey:'[DocumentField(value_type=dictionary, value={'satis': DocumentField(value_type=string, value='V', content=V, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=4.515, y=8.105), Point(x=4.645, y=8.105), Point(x=4.645, y=8.34), Point(x=4.515, y=8.34)])], spans=[DocumentSpan(offset=218, length=1)], confidence=None), 'dissatis': DocumentField(value_type=string, value=None, content=None, bounding_regions=[], spans=[], confidence=None)}, content=None, bounding_regions=[], spans=[], confidence=None), DocumentField(value_type=dictionary, value={'satis': DocumentField(value_type=string, value='V', content=V, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=4.525, y=8.69), Point(x=4.655, y=8.69), Point(x=4.655, y=8.925), Point(x=4.525, y=8.925)])], spans=[DocumentSpan(offset=234, length=1)], confidence=None), 'dissatis': DocumentField(value_type=string, value=None, content=None, bounding_regions=[], spa

In [60]:
import re

# 파일 경로 설정
txt_file_path = "C:/Users/parkm/Desktop/예제1.txt"

# 정보 추출을 위한 정규식 패턴
information_pattern = r"information:'({.*?})'"
satis_survey_pattern = r"satis_survey:'\[(.*?)\]'"

# 파일 내용 읽기
with open(txt_file_path, "r") as file:
    memo = file.read()

#reason, learnt 줄은 그대로 유지. 건너띄기. information줄에선 information:만 남기고 뒤에 내용 아래 수행 정보로 대체
satis_survey 줄에서도 information줄처럼 해!!!!!!!!!!!!!!!!!

# 정보 추출
information_data = {}
satis_survey_data = []

# 정보 추출
information_match = re.search(information_pattern, memo, re.DOTALL)
if information_match:
    information_section = information_match.group(1)
    for match in re.finditer(r"'(\w+)': DocumentField.*?value='(.*?)'", information_section):
        key = match.group(1)
        value = match.group(2)
        information_data[key] = value

# 만약 satis_survey가 존재하면 추출
satis_survey_match = re.search(satis_survey_pattern, memo, re.DOTALL)
if satis_survey_match:
    satis_survey_section = satis_survey_match.group(1)
    for match in re.finditer(r"'satis':\s*DocumentField.*?value='(.*?)'", satis_survey_section):
        satis_survey_data.append({'satis': match.group(1)})

# 파일 내용 수정
with open(txt_file_path, "w") as file:
    file.write(f"information: {information_data}\n")
    file.write(f"satis_survey: {satis_survey_data}\n")

print("텍스트 파일이 성공적으로 수정되었습니다.")


텍스트 파일이 성공적으로 수정되었습니다.
